In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import urllib.request
from urllib.parse import quote
import os
import http.cookiejar
import json
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def get_soup(url,header):
    """Get result as html and return is parsed by bs4"""
    request = urllib.request.Request(url,headers=header)
    request = urllib.request.urlopen(request)
    return BeautifulSoup(request,'html.parser')

In [ ]:
def make_request(first):
    url="https://www.bing.com/images/async?q="+format(quote(query))+"&first="+str(first)+"&count=100&relp=35&scenario=ImageBasicHover&datsrc=N_I&layout=RowBased&mmasync=1&dgState=x*255_y*963_h*182_c*1_i*246_r*40&IG=DFAB70E8D3754C3194E80C53E3650DAE&SFX=7&iid=images.5948"
    soup = get_soup(url,header)
    print (url)
    return soup

In [ ]:
query = "face"# you can change the query for the image  here
image_type="faces" # name for the image
DIR="Pictures" # directory
query= query.split()
query='+'.join(query)
header={'User-Agent':"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:57.0) Gecko/20100101 Firefox/57.0"}
actualImages=[] # contains the links for images

In [ ]:
# making directory for images
if not os.path.exists(DIR):
            os.mkdir(DIR)
DIR = os.path.join(DIR, query.split()[0])

if not os.path.exists(DIR):
            os.mkdir(DIR)

In [ ]:
def itter_result(soup):
    """a function for getting array of links from query result"""
    for a in soup.find_all("a",{"class":"iusc"}):
        link =json.loads(a.attrs["m"])["murl"]
        print (link)
        actualImages.append(link)
    print  ("there are total" , len(actualImages),"images")

In [ ]:
for i in range(0, 1000, 100):
    soup =  make_request(i)
    itter_result(soup)

In [ ]:
"""loading haar cascade to detect the desired objects in the image"""
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml') 
# cat_cascade = cv.CascadeClassifier('haarcascade_frontalcatface.xml') 
if face_cascade.empty():
    print  ('Cascade loading error!')

In [ ]:
for i , img  in enumerate(actualImages):
    """loading images and detection objects via haar cascade"""
    try:
        req = urllib.request.Request(img, headers=header)
        raw_img = urllib.request.urlopen(req, timeout=1).read()
        
        image = np.asarray(bytearray(raw_img), dtype="uint8") #converting image to np array of unsigned integer
        image = cv.imdecode(image, cv.IMREAD_COLOR)
        gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5) #detecting objects
        
        for (x,y,w,h) in faces:
            roi_color = image[y:y+h, x:x+w]
            cntr = len([file for file in os.listdir(DIR) if image_type in file]) + 1
            print (cntr)
            cv.imwrite((DIR+'/'+image_type + "_"+ str(cntr)+".jpg"),roi_color) 
            
    except Exception as e:
        print ("could not load : "+img)
        print (e)